## 1. Load Data and Explore Variables

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load hourly data
data_path = Path('../data/processed/hourly/all_locations_hourly.parquet')
hourly = pd.read_parquet(data_path)

print(f"Dataset shape: {hourly.shape}")
print(f"Total variables: {len(hourly.columns)}")
print(f"\nAll columns:")
print(hourly.columns.tolist())

Dataset shape: (263160, 33)
Total variables: 33

All columns:
['date', 'location_code', 'location_name', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'weather_code', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'et0_fao_evapotranspiration', 'vapour_pressure_deficit', 'wind_gusts_10m', 'wind_direction_100m', 'wind_direction_10m', 'wind_speed_100m', 'wind_speed_10m', 'soil_temperature_0_to_7cm', 'soil_temperature_7_to_28cm', 'soil_temperature_28_to_100cm', 'soil_temperature_100_to_255cm', 'soil_moisture_0_to_7cm', 'soil_moisture_7_to_28cm', 'soil_moisture_28_to_100cm', 'soil_moisture_100_to_255cm']


## 2. Categorize Variables by Relevance

In [2]:
# CATEGORY 1: Ground-Level Weather (What tourists feel) - KEEP
ground_level_vars = [
    'temperature_2m',              # Surface temperature (tourists feel this!)
    'relative_humidity_2m',        # Humidity at 2m (affects comfort)
    'dew_point_2m',               # Dew point (related to comfort)
    'apparent_temperature',        # Feels-like temperature (very important!)
    'precipitation',               # Rain amount (tourists avoid rain)
    'rain',                        # Rainfall
    'weather_code',                # Weather condition code
    'wind_speed_10m',              # Wind at 10m height
    'wind_direction_10m',          # Wind direction
    'wind_gusts_10m',              # Wind gusts (safety)
    'pressure_msl',                # Sea level pressure
    'surface_pressure',            # Surface pressure
    'cloud_cover',                 # Total cloud cover
    'cloud_cover_low',             # Low clouds
    'cloud_cover_mid',             # Mid clouds
    'cloud_cover_high',            # High clouds
    'visibility',                  # Visibility (for activities)
    'vapour_pressure_deficit'      # Related to comfort/dryness
]

# CATEGORY 2: High-Altitude/Atmospheric (Tourists DON'T feel) - REMOVE
atmospheric_vars = [
    # Temperature at various pressure levels (altitude)
    'temperature_1000hPa', 'temperature_975hPa', 'temperature_950hPa',
    'temperature_925hPa', 'temperature_900hPa', 'temperature_850hPa',
    'temperature_800hPa', 'temperature_700hPa', 'temperature_600hPa',
    'temperature_500hPa', 'temperature_400hPa', 'temperature_300hPa',
    'temperature_250hPa', 'temperature_200hPa', 'temperature_150hPa',
    'temperature_100hPa', 'temperature_70hPa', 'temperature_50hPa', 'temperature_30hPa',
    
    # Humidity at various pressure levels
    'relative_humidity_1000hPa', 'relative_humidity_975hPa', 'relative_humidity_950hPa',
    'relative_humidity_925hPa', 'relative_humidity_900hPa', 'relative_humidity_850hPa',
    'relative_humidity_800hPa', 'relative_humidity_700hPa', 'relative_humidity_600hPa',
    'relative_humidity_500hPa', 'relative_humidity_400hPa', 'relative_humidity_300hPa',
    'relative_humidity_250hPa', 'relative_humidity_200hPa', 'relative_humidity_150hPa',
    'relative_humidity_100hPa', 'relative_humidity_70hPa', 'relative_humidity_50hPa', 'relative_humidity_30hPa',
    
    # Cloud cover at various pressure levels
    'cloud_cover_1000hPa', 'cloud_cover_975hPa', 'cloud_cover_950hPa',
    'cloud_cover_925hPa', 'cloud_cover_900hPa', 'cloud_cover_850hPa',
    'cloud_cover_800hPa', 'cloud_cover_700hPa', 'cloud_cover_600hPa',
    'cloud_cover_500hPa', 'cloud_cover_400hPa', 'cloud_cover_300hPa',
    'cloud_cover_250hPa', 'cloud_cover_200hPa', 'cloud_cover_150hPa',
    'cloud_cover_100hPa', 'cloud_cover_70hPa', 'cloud_cover_50hPa', 'cloud_cover_30hPa',
    
    # Wind at various pressure levels
    'wind_speed_1000hPa', 'wind_speed_975hPa', 'wind_speed_950hPa',
    'wind_speed_925hPa', 'wind_speed_900hPa', 'wind_speed_850hPa',
    'wind_speed_800hPa', 'wind_speed_700hPa', 'wind_speed_600hPa',
    'wind_speed_500hPa', 'wind_speed_400hPa', 'wind_speed_300hPa',
    'wind_speed_250hPa', 'wind_speed_200hPa', 'wind_speed_150hPa',
    'wind_speed_100hPa', 'wind_speed_70hPa', 'wind_speed_50hPa', 'wind_speed_30hPa',
    
    'wind_direction_1000hPa', 'wind_direction_975hPa', 'wind_direction_950hPa',
    'wind_direction_925hPa', 'wind_direction_900hPa', 'wind_direction_850hPa',
    'wind_direction_800hPa', 'wind_direction_700hPa', 'wind_direction_600hPa',
    'wind_direction_500hPa', 'wind_direction_400hPa', 'wind_direction_300hPa',
    'wind_direction_250hPa', 'wind_direction_200hPa', 'wind_direction_150hPa',
    'wind_direction_100hPa', 'wind_direction_70hPa', 'wind_direction_50hPa', 'wind_direction_30hPa',
    
    # Geopotential heights (completely irrelevant for tourists)
    'geopotential_height_1000hPa', 'geopotential_height_975hPa', 'geopotential_height_950hPa',
    'geopotential_height_925hPa', 'geopotential_height_900hPa', 'geopotential_height_850hPa',
    'geopotential_height_800hPa', 'geopotential_height_700hPa', 'geopotential_height_600hPa',
    'geopotential_height_500hPa', 'geopotential_height_400hPa', 'geopotential_height_300hPa',
    'geopotential_height_250hPa', 'geopotential_height_200hPa', 'geopotential_height_150hPa',
    'geopotential_height_100hPa', 'geopotential_height_70hPa', 'geopotential_height_50hPa', 'geopotential_height_30hPa',
    
    # High altitude temperatures and winds
    'temperature_80m', 'temperature_120m', 'temperature_180m',
    'wind_speed_80m', 'wind_speed_120m', 'wind_speed_180m',
    'wind_direction_80m', 'wind_direction_120m', 'wind_direction_180m'
]

# CATEGORY 3: Optional/Nice-to-Have - EVALUATE
optional_vars = [
    'snowfall',                    # Rare in SA (might remove)
    'snow_depth',                  # Rare in SA (might remove)
    'showers',                     # Similar to precipitation (might remove)
    'evapotranspiration',          # Might be useful
    'et0_fao_evapotranspiration',  # Might be useful
    'soil_moisture_0_to_1cm',      # Might remove (not directly relevant)
    'soil_moisture_1_to_3cm',
    'soil_moisture_3_to_9cm',
    'soil_moisture_9_to_27cm',
    'soil_moisture_27_to_81cm',
    'soil_temperature_0cm',        # Might remove
    'soil_temperature_6cm',
    'soil_temperature_18cm',
    'soil_temperature_54cm',
    'precipitation_probability'    # If available in hourly
]

# CATEGORY 4: Metadata (Always keep)
metadata_vars = [
    'date',
    'location_code',
    'location_name'
]

print(f"Ground-level variables: {len(ground_level_vars)}")
print(f"Atmospheric variables (to remove): {len(atmospheric_vars)}")
print(f"Optional variables: {len(optional_vars)}")
print(f"Metadata variables: {len(metadata_vars)}")

Ground-level variables: 18
Atmospheric variables (to remove): 123
Optional variables: 15
Metadata variables: 3


## 3. Check Which Variables Exist in Our Data

In [3]:
# Check which ground-level vars we actually have
available_ground = [v for v in ground_level_vars if v in hourly.columns]
missing_ground = [v for v in ground_level_vars if v not in hourly.columns]

print(f"Available ground-level vars: {len(available_ground)}")
print(available_ground)
print(f"\nMissing ground-level vars: {len(missing_ground)}")
print(missing_ground)

Available ground-level vars: 17
['temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'weather_code', 'wind_speed_10m', 'wind_direction_10m', 'wind_gusts_10m', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'vapour_pressure_deficit']

Missing ground-level vars: 1
['visibility']


## 4. Select Final Variables

In [4]:
# Final selection: Metadata + Ground-level + Useful optional
selected_vars = metadata_vars + available_ground

# Add useful optional vars if they exist
useful_optional = ['evapotranspiration', 'et0_fao_evapotranspiration', 'precipitation_probability']
for var in useful_optional:
    if var in hourly.columns and var not in selected_vars:
        selected_vars.append(var)

# Create filtered dataset
hourly_filtered = hourly[selected_vars].copy()

print(f"\n{'='*80}")
print(f"FINAL HOURLY VARIABLE SELECTION")
print(f"{'='*80}")
print(f"Original variables: {len(hourly.columns)}")
print(f"Selected variables: {len(selected_vars)}")
print(f"Removed variables: {len(hourly.columns) - len(selected_vars)}")
print(f"\nSelected variables:")
for i, var in enumerate(selected_vars, 1):
    print(f"  {i:2d}. {var}")


FINAL HOURLY VARIABLE SELECTION
Original variables: 33
Selected variables: 21
Removed variables: 12

Selected variables:
   1. date
   2. location_code
   3. location_name
   4. temperature_2m
   5. relative_humidity_2m
   6. dew_point_2m
   7. apparent_temperature
   8. precipitation
   9. rain
  10. weather_code
  11. wind_speed_10m
  12. wind_direction_10m
  13. wind_gusts_10m
  14. pressure_msl
  15. surface_pressure
  16. cloud_cover
  17. cloud_cover_low
  18. cloud_cover_mid
  19. cloud_cover_high
  20. vapour_pressure_deficit
  21. et0_fao_evapotranspiration


## 5. Check for Missing Values

In [5]:
# Check missing values in selected variables
missing = hourly_filtered.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print("\nVariables with missing values:")
    print(missing)
    print(f"\nPercentage missing:")
    print((missing / len(hourly_filtered) * 100).round(2))
else:
    print("\n✅ No missing values in selected variables!")


✅ No missing values in selected variables!


## 6. Summary Statistics

In [6]:
# Summary stats for numeric variables
numeric_vars = hourly_filtered.select_dtypes(include=[np.number]).columns
hourly_filtered[numeric_vars].describe()

,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,weather_code,wind_speed_10m,wind_direction_10m,wind_gusts_10m,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,vapour_pressure_deficit,et0_fao_evapotranspiration
count,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000,263160.000000
mean,17.683614,67.925687,10.684321,16.622628,0.086529,0.086522,8.279628,12.325908,177.809948,28.320169,1017.873721,960.057484,40.360108,20.901258,15.860572,20.011848,0.759648,0.157806
std,5.676702,21.498506,6.352319,6.897315,0.439549,0.439531,18.126650,7.377096,104.871480,15.188042,5.893092,64.957942,40.453615,34.019409,28.327946,34.568303,0.726961,0.196286
min,-4.018000,4.091573,-22.630999,-10.080518,0.000000,0.000000,0.000000,0.000000,0.530492,1.440000,996.800000,821.709200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.952000,53.455986,7.410500,11.978917,0.000000,0.000000,0.000000,6.952755,84.289500,16.560000,1013.700000,877.972660,0.000000,0.000000,0.000000,0.000000,0.243301,0.008749
50%,17.652000,71.745655,11.665501,16.592491,0.000000,0.000000,1.000000,10.630672,177.754300,25.560000,1017.400000,999.500270,27.000000,0.000000,0.000000,0.000000,0.534672,0.059808
75%,21.310501,85.754837,15.051001,21.142641,0.000000,0.000000,3.000000,16.179987,266.633600,36.719997,1021.700000,1011.705455,87.000000,28.000000,19.000000,26.000000,1.030685,0.267054
max,41.060500,100.000000,26.302000,43.415770,20.800000,20.800000,73.000000,61.921043,360.000000,138.599990,1040.500000,1039.379900,100.000000,100.000000,100.000000,100.000000,7.193762,0.996771


## 7. Save Filtered Dataset

In [7]:
# Save filtered hourly data
output_path = Path('../data/processed/hourly/hourly_filtered.parquet')
hourly_filtered.to_parquet(output_path, index=False)

print(f"\n✅ Filtered hourly data saved to: {output_path}")
print(f"   Shape: {hourly_filtered.shape}")
print(f"   File size: {output_path.stat().st_size / (1024**2):.2f} MB")


✅ Filtered hourly data saved to: ..\data\processed\hourly\hourly_filtered.parquet
   Shape: (263160, 21)
   File size: 14.35 MB


## 8. Export Variable List

In [8]:
# Save selected variable names for documentation
selected_vars_df = pd.DataFrame({
    'variable': selected_vars,
    'category': ['metadata'] * 3 + ['ground_level'] * (len(selected_vars) - 3)
})

selected_vars_df.to_csv('../docs/selected_hourly_variables.csv', index=False)
print("\n✅ Variable list saved to: docs/selected_hourly_variables.csv")


✅ Variable list saved to: docs/selected_hourly_variables.csv
